## ArtificiaL Neural Network

Importing libraries

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

### Data Preprocessing

In [ ]:
df = pd.read_csv(#data.csv)
df.head()

X = df.iloc[:,3:-1].values
y = df.iloc[:,-1].values

Encoding categorica data

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,2] = le.fit_transform(X[:,2])

One Hot Encoding for features with multiple categories

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[("encoder", OneHotEncoder(), [1])], remainder="passthrough")
X = np.array(ct.fit_transform(X))

Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### Building the ANN

Initializing

In [ ]:
ann = tf.keras.models.Sequential()

Adding the input layer and first hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=6, activation="relu"))

Adding the second hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=6, activation="relu"))

Adding the output layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

### Training the ANN

Compiling

In [ ]:
ann.compile(optimizer= "adam", loss= "binary_crossentropy", metrics= ["accuracy"])

Training

In [ ]:
ann.fit(X_train, y_train, batch_size= 32, epochs= 100)

### Model Evaluation

In [ ]:
y_pred = ann.predict(X_test)

y_pred = (y_pred > 0.5)

np.concatenate((y_pred.reshape(-1, 1), y_test.reshape(-1, 1)), axis= 1)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
confusion_matrix(y_test, y_pred)
accuracy_score(y_test, y_pred)

